In [2]:
from IPython.display import clear_output
!pip install tensorflow_addons
clear_output()

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from modules.dataset import LABELS, Dataset

2023-01-04 08:06:42.364751: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Functions

In [4]:
def get_test_dataset_5_fold():
    dataset = Dataset()
    _, test_dataset_fold_1 = dataset.get_kfold(fold_number=1, sample=False)
    _, test_dataset_fold_2 = dataset.get_kfold(fold_number=2, sample=False)
    _, test_dataset_fold_3 = dataset.get_kfold(fold_number=3, sample=False)
    _, test_dataset_fold_4 = dataset.get_kfold(fold_number=4, sample=False)
    _, test_dataset_fold_5 = dataset.get_kfold(fold_number=5, sample=False)
    return (
        test_dataset_fold_1,
        test_dataset_fold_2,
        test_dataset_fold_3,
        test_dataset_fold_4,
        test_dataset_fold_5
    )

In [5]:
def get_y_true(data):
    y_true=[]
    for X,y in data:
        for label in y:
            y_true.append(label)
    y_true = tf.Variable(y_true)
    return y_true

In [6]:
def get_model(model_path):
    return tf.keras.models.load_model(model_path)

In [7]:
def get_filename(model_path):
    return model_path.split(".")[0].split("/")[-1]

In [8]:
def model_predict(model, test_dataset):
    return model.predict(test_dataset)

In [9]:
def evaluate(model, test_dataset, file_name):
    y_true = get_y_true(test_dataset)
    y_preds = model_predict(model, test_dataset)
    m = tf.keras.metrics.AUC(multi_label=True)
    m.update_state(y_true, y_preds)
    return m.result().numpy()

In [10]:
def get_model_path(model_name, fold):
    model_path = {
        "DenseNet121_none": "/home/jovyan/ChestXray-14/results/models/DenseNet121_None_fold_{}.h5",
        "DenseNet121_imagenet": "/home/jovyan/ChestXray-14/results/models/DenseNet121_imagenet_fold_{}.h5",
        "EfficeientNetB0_none": "/home/jovyan/ChestXray-14/results/models/EfficientNetB0_None_fold_{}.h5",
        "EfficeientNetB0_imagenet": "/home/jovyan/ChestXray-14/results/models/EfficientNetB0_imagenet_fold_{}.h5",
        "Resnet50_none": "/home/jovyan/ChestXray-14/results/models/Resnet50_fold_{}.h5",
        "Resnet50_imagenet": "/home/jovyan/ChestXray-14/results/models/Resnet50_imagenet_fold_{}.h5"
    }
    return model_path[model_name].format(fold)

In [11]:
def create_dataframe(auc_5_folds, file_name):
    df = pd.DataFrame({"AUC": auc_5_folds})
    df.to_csv(f"/home/jovyan/ChestXray-14/results/paper/table2/{file_name}.csv", index=False)

In [12]:
def evaluate_5_folds(model_name):
    """
    Parameter
    ---------
    path: str
        path to best model
    """
    auc_5_folds = []
    for i in range(5): # 5 Folds
        path = get_model_path(model_name, fold=i+1)
        model = get_model(path)
        file_name = get_filename(path)
        auc = evaluate(model=model, test_dataset=test_dataset[i], file_name=file_name)
        auc_5_folds.append(auc)
        print(f"===== {file_name} =====")
        print(f">> Fold {i+1} = {auc}")
        # break
    
    print(f"Average AUC of {file_name} = {np.mean(auc_5_folds)}")
    create_dataframe(auc_5_folds, file_name)

## Main

In [25]:
test_dataset = get_test_dataset_5_fold()

In [26]:
evaluate_5_folds("DenseNet121_none")

1402/1402 [==============================] - 57s 39ms/step
===== DenseNet121_None_fold_1 =====
>> Fold 1 = 0.7168176174163818
1402/1402 [==============================] - 56s 38ms/step
===== DenseNet121_None_fold_2 =====
>> Fold 2 = 0.7234813570976257
1402/1402 [==============================] - 54s 37ms/step
===== DenseNet121_None_fold_3 =====
>> Fold 3 = 0.7426453232765198
1402/1402 [==============================] - 55s 38ms/step
===== DenseNet121_None_fold_4 =====
>> Fold 4 = 0.7323508858680725
1402/1402 [==============================] - 54s 37ms/step
===== DenseNet121_None_fold_5 =====
>> Fold 5 = 0.7227852940559387
Average AUC of DenseNet121_None_fold_5 = 0.7276160717010498


In [27]:
evaluate_5_folds("DenseNet121_imagenet")

1402/1402 [==============================] - 56s 38ms/step
===== DenseNet121_imagenet_fold_1 =====
>> Fold 1 = 0.7416366934776306
1402/1402 [==============================] - 56s 38ms/step
===== DenseNet121_imagenet_fold_2 =====
>> Fold 2 = 0.7446348071098328
1402/1402 [==============================] - 56s 38ms/step
===== DenseNet121_imagenet_fold_3 =====
>> Fold 3 = 0.7446626424789429
1402/1402 [==============================] - 57s 40ms/step
===== DenseNet121_imagenet_fold_4 =====
>> Fold 4 = 0.7416130304336548
1402/1402 [==============================] - 56s 38ms/step
===== DenseNet121_imagenet_fold_5 =====
>> Fold 5 = 0.7413163781166077
Average AUC of DenseNet121_imagenet_fold_5 = 0.7427726984024048


In [28]:
evaluate_5_folds("EfficeientNetB0_none")

1402/1402 [==============================] - 47s 30ms/step
===== EfficientNetB0_None_fold_1 =====
>> Fold 1 = 0.7434552907943726
1402/1402 [==============================] - 42s 29ms/step
===== EfficientNetB0_None_fold_2 =====
>> Fold 2 = 0.7409191131591797
1402/1402 [==============================] - 42s 29ms/step
===== EfficientNetB0_None_fold_3 =====
>> Fold 3 = 0.7506392598152161
1402/1402 [==============================] - 42s 29ms/step
===== EfficientNetB0_None_fold_4 =====
>> Fold 4 = 0.7430868744850159
1402/1402 [==============================] - 43s 29ms/step
===== EfficientNetB0_None_fold_5 =====
>> Fold 5 = 0.738027811050415
Average AUC of EfficientNetB0_None_fold_5 = 0.7432256937026978


In [29]:
evaluate_5_folds("EfficeientNetB0_imagenet")

1402/1402 [==============================] - 42s 29ms/step
===== EfficientNetB0_imagenet_fold_1 =====
>> Fold 1 = 0.6801937222480774
1402/1402 [==============================] - 43s 29ms/step
===== EfficientNetB0_imagenet_fold_2 =====
>> Fold 2 = 0.49319955706596375
1402/1402 [==============================] - 42s 29ms/step
===== EfficientNetB0_imagenet_fold_3 =====
>> Fold 3 = 0.626673698425293
1402/1402 [==============================] - 43s 29ms/step
===== EfficientNetB0_imagenet_fold_4 =====
>> Fold 4 = 0.54878169298172
1402/1402 [==============================] - 42s 29ms/step
===== EfficientNetB0_imagenet_fold_5 =====
>> Fold 5 = 0.6150543093681335
Average AUC of EfficientNetB0_imagenet_fold_5 = 0.592780590057373


In [30]:
evaluate_5_folds("Resnet50_none")

1402/1402 [==============================] - 40s 27ms/step
===== Resnet50_fold_1 =====
>> Fold 1 = 0.7188717126846313
1402/1402 [==============================] - 37s 26ms/step
===== Resnet50_fold_2 =====
>> Fold 2 = 0.7164515256881714
1402/1402 [==============================] - 37s 26ms/step
===== Resnet50_fold_3 =====
>> Fold 3 = 0.7254135012626648
1402/1402 [==============================] - 37s 26ms/step
===== Resnet50_fold_4 =====
>> Fold 4 = 0.7192507386207581
1402/1402 [==============================] - 37s 26ms/step
===== Resnet50_fold_5 =====
>> Fold 5 = 0.7044130563735962
Average AUC of Resnet50_fold_5 = 0.7168801426887512


In [31]:
evaluate_5_folds("Resnet50_imagenet")

1402/1402 [==============================] - 37s 26ms/step
===== Resnet50_imagenet_fold_1 =====
>> Fold 1 = 0.7305748462677002
1402/1402 [==============================] - 37s 26ms/step
===== Resnet50_imagenet_fold_2 =====
>> Fold 2 = 0.7403049468994141
1402/1402 [==============================] - 38s 26ms/step
===== Resnet50_imagenet_fold_3 =====
>> Fold 3 = 0.7430562973022461
1402/1402 [==============================] - 37s 26ms/step
===== Resnet50_imagenet_fold_4 =====
>> Fold 4 = 0.7355892062187195
1402/1402 [==============================] - 37s 26ms/step
===== Resnet50_imagenet_fold_5 =====
>> Fold 5 = 0.7230795621871948
Average AUC of Resnet50_imagenet_fold_5 = 0.7345210313796997
